# EDA Seguro Médico

Notebook de análisis exploratorio de datos (EDA) sobre el dataset de costos de seguros médicos.
Utilizamos métodos de limpieza, transformación y análisis para preparar los datos para modelado.

## 1. Carga de librerías y datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

# Configuración de visualizaciones
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

DATA_DIR = 'data'
OUTPUT_DIR = 'output'
FIGS_DIR = os.path.join(OUTPUT_DIR, 'figs')
os.makedirs(FIGS_DIR, exist_ok=True)

# Carga de datos
df = pd.read_csv(os.path.join(DATA_DIR, 'insurance.csv'))
print(f'Dataset cargado: {df.shape[0]} registros, {df.shape[1]} columnas')
df.head()

## 2. Análisis inicial y descripción del conjunto de datos
Revisamos los tipos de variables, datos faltantes y estadísticas básicas para entender la estructura del dataset.

In [ ]:
print('=== INFORMACIÓN DEL DATASET ===')
df.info()

In [ ]:
print('\n=== VALORES FALTANTES ===')
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else 'No hay valores faltantes ✓')

In [ ]:
print('\n=== ESTADÍSTICAS DESCRIPTIVAS ===')
df.describe(include='all').round(2)

> **Justificación**: Revisar tipos y valores faltantes asegura que el análisis sea confiable y permite decidir si se requiere imputación o limpieza adicional. En este caso, no hay valores faltantes, lo cual facilita el análisis.

## 3. Transformaciones iniciales
Convertimos columnas categóricas al tipo 'category' para optimizar memoria y facilitar análisis posteriores.

In [ ]:
cat_cols = ['sex', 'smoker', 'region']
for col in cat_cols:
    df[col] = df[col].astype('category')

print('✓ Columnas categóricas convertidas:', cat_cols)
print(df.dtypes)

## 4. Análisis univariante: Variables numéricas
Examinamos la distribución de cada variable numérica individualmente.

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Variables numéricas: {num_cols}\n')

for col in num_cols:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.histplot(df[col], kde=True, ax=ax, bins=30, color='steelblue')
    ax.set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Frecuencia')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'01_univariate_{col}.png'), dpi=100)
    plt.show()
    print(f'{col}: Min={df[col].min():.2f}, Max={df[col].max():.2f}, Media={df[col].mean():.2f}')

## 5. Análisis univariante: Variables categóricas
Revisamos la frecuencia de categorías en cada variable.

In [ ]:
for col in cat_cols + ['children']:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.countplot(data=df, x=col, ax=ax, palette='Set2')
    ax.set_title(f'Conteo por {col}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Cantidad')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'02_univariate_{col}.png'), dpi=100)
    plt.show()
    print(f'\n{col}:')
    print(df[col].value_counts())

## 📊 Deducciones univariantes

**age**: Distribución aproximadamente uniforme. Los pacientes tienen edades entre 18 y 64 años, lo que sugiere una muestra representativa de diferentes grupos etarios.

**bmi**: Distribución casi normal centrada alrededor de 30. La mayoría de pacientes tienen BMI normal a sobrepeso.

**charges** (Variable Objetivo): Distribución fuertemente sesgada a la derecha. Hay muchos pacientes con costos bajos pero algunos con costos extremadamente altos (hasta $63,770), lo que indica la presencia de outliers.

**smoker**: Desequilibrio claro. Aproximadamente el 20% son fumadores y el 80% no fumadores. Esta proporción desigual es importante para la modelización.

**sex**: Distribución casi equilibrada entre hombres (50.5%) y mujeres (49.5%).

**region**: Distribución uniforme entre las 4 regiones geográficas (southwest, southeast, northwest, northeast), cada una con ~25% de los datos.

**children**: Mayor concentración en 0 hijos (42.9%), decrece con el número de hijos. Pocos pacientes tienen 5 hijos.

## 6. Filtrado de outliers (método IQR)
Eliminamos valores extremos en la variable objetivo (charges) usando el método de intercuartiles.

In [ ]:
def remove_outliers_iqr(df_in, col, k=1.5):
    """Elimina outliers usando el método IQR.
    k=1.5 es el estándar; k=3.0 es más conservador.
    """
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    
    print(f'Q1={q1:.2f}, Q3={q3:.2f}, IQR={iqr:.2f}')
    print(f'Límite inferior: {lower_bound:.2f}')
    print(f'Límite superior: {upper_bound:.2f}')
    
    mask = (df_in[col] >= lower_bound) & (df_in[col] <= upper_bound)
    outliers_removed = (~mask).sum()
    print(f'Outliers eliminados: {outliers_removed}')
    
    return df_in.loc[mask]

print(f'Dataset original: {df.shape[0]} registros')
df_filtered = remove_outliers_iqr(df, 'charges', k=1.5)
print(f'Dataset después de filtrar: {df_filtered.shape[0]} registros')
print(f'Registros eliminados: {df.shape[0] - df_filtered.shape[0]} ({((df.shape[0] - df_filtered.shape[0])/df.shape[0]*100):.1f}%)')

> **Justificación**: El método IQR (Interquartile Range) es robusto y estadísticamente fundamentado. Elimina valores extremos que pueden distorsionar los modelos, permitiendo trabajar con datos más representativos. Usamos k=1.5 (estándar) para identificar outliers moderados.

## 7. Tratamiento de la variable objetivo
Decidimos si aplicar transformación logarítmica a 'charges' para mejorar la distribución.

In [ ]:
print('=== ANÁLISIS DE CHARGES ===' )
print(f'Mín: ${df_filtered["charges"].min():,.2f}')
print(f'Máx: ${df_filtered["charges"].max():,.2f}')
print(f'Media: ${df_filtered["charges"].mean():,.2f}')
print(f'Mediana: ${df_filtered["charges"].median():,.2f}')
print(f'Asimetría (Skewness): {df_filtered["charges"].skew():.2f}')
print(f'Curtosis: {df_filtered["charges"].kurtosis():.2f}')

# Aplicar transformación log
if (df_filtered['charges'] > 0).all():
    df_filtered['log_charges'] = np.log(df_filtered['charges'])
    print('\n✓ Transformación logarítmica aplicada a charges')
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.histplot(df_filtered['charges'], kde=True, ax=axes[0], color='steelblue', bins=30)
    axes[0].set_title('Charges - Original', fontweight='bold')
    axes[0].set_xlabel('Charges ($)')
    
    sns.histplot(df_filtered['log_charges'], kde=True, ax=axes[1], color='coral', bins=30)
    axes[1].set_title('Charges - Log Transformado', fontweight='bold')
    axes[1].set_xlabel('Log(Charges)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, '03_charges_transformation.png'), dpi=100)
    plt.show()
else:
    print('⚠ Advertencia: charges tiene valores ≤ 0; no se puede aplicar log sin ajuste')

> **Justificación**: La transformación logarítmica normaliza la distribución sesgada de charges, mejorando el rendimiento de modelos lineales y facilitando la interpretación de relaciones exponenciales.

## 8. Análisis bivariante: Variables numéricas vs Charges
Examinamos la relación entre cada variable numérica y la variable objetivo.

In [ ]:
numeric_features = [col for col in num_cols if col not in ['charges', 'log_charges']]

for col in numeric_features:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.scatterplot(data=df_filtered, x=col, y='charges', alpha=0.5, ax=ax, color='steelblue')
    
    # Calcular correlación
    corr = df_filtered[col].corr(df_filtered['charges'])
    
    ax.set_title(f'Charges vs {col} (Correlación: {corr:.3f})', fontsize=12, fontweight='bold')
    ax.set_ylabel('Charges ($)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'04_bivariate_{col}_vs_charges.png'), dpi=100)
    plt.show()

## 9. Análisis bivariante: Variables categóricas vs Charges
Comparamos cómo cada categoría impacta el costo del seguro.

In [ ]:
for col in cat_cols + ['children']:
    fig, ax = plt.subplots(1, 1, figsize=(9, 5))
    sns.boxplot(data=df_filtered, x=col, y='charges', ax=ax, palette='Set2')
    ax.set_title(f'Distribución de Charges por {col}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Charges ($)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'05_bivariate_{col}.png'), dpi=100)
    plt.show()

## 📊 Deducciones bivariantes

**age vs charges**: Relación cuadrática clara. Los costos permanecen bajos hasta ~45 años, luego aumentan drásticamente. Esto sugiere mayor consumo de servicios médicos con la edad.

**bmi vs charges**: Correlación positiva moderada. Personas con BMI más alto tienden a tener costos más altos, sugiriendo que sobrepeso/obesidad aumenta riesgo médico.

**children vs charges**: Relación débil. El número de hijos tiene poco impacto en el costo individual del paciente.

**smoker vs charges**: Diferencia MUY significativa. Los fumadores pagan aproximadamente 3-4 veces más que no fumadores. Esta es la variable categórica más importante.

**sex vs charges**: Diferencias mínimas entre hombres y mujeres. Esta variable tiene poco poder predictivo.

**region vs charges**: Diferencias muy pequeñas entre regiones geográficas. Región no es factor relevante en costos.

## 10. Matriz de correlación
Identificamos relaciones lineales entre todas las variables numéricas.

In [ ]:
# Calcular correlaciones solo con variables numéricas
corr_matrix = df_filtered.select_dtypes(include=[np.number]).corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlación - Variables Numéricas', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, '06_correlation_matrix.png'), dpi=100)
plt.show()

print('\n=== CORRELACIONES CON CHARGES ===')
correlations = corr_matrix['charges'].sort_values(ascending=False)
print(correlations)

## 📊 Análisis de Correlaciones

**Correlaciones Positivas Altas:**
- **age (0.60)**: Relación fuerte. A mayor edad, mayor costo.
- **log_charges (1.00)**: Perfecta (es transformación de charges).

**Correlaciones Moderadas:**
- **bmi (0.41)**: Relación moderada. BMI más alto = costos más altos.

**Correlaciones Débiles:**
- **children (-0.08)**: Prácticamente no correlacionada.

**Decisión de Variables:**
- **Mantener**: age, bmi (ambas tienen poder predictivo claro)
- **Revisar**: children (muy baja correlación, pero incluir para contexto)
- **Nota**: Las categorías smoker, sex y region se evaluarán en análisis posterior (requieren encoding)

## 11. División del dataset: Train (80%) y Test (20%)
Estratificamos por bins de charges para mantener la distribución de la variable objetivo.

In [ ]:
# Crear bins para estratificación
n_bins = 5
df_filtered['charges_bin'] = pd.qcut(df_filtered['charges'], q=n_bins, labels=False, duplicates='drop')

print(f'Bins creados: {df_filtered["charges_bin"].nunique()}')
print(f'\nDistribución por bin:\n{df_filtered["charges_bin"].value_counts().sort_index()}')

# Split estratificado
train_df, test_df = train_test_split(
    df_filtered, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_filtered['charges_bin']
)

# Remover columna auxiliar
train_df = train_df.drop(columns=['charges_bin', 'log_charges'])
test_df = test_df.drop(columns=['charges_bin', 'log_charges'])

print(f'\nTrain set: {train_df.shape[0]} registros ({train_df.shape[0]/df_filtered.shape[0]*100:.1f}%)')
print(f'Test set: {test_df.shape[0]} registros ({test_df.shape[0]/df_filtered.shape[0]*100:.1f}%)')

## 12. Verificación de estratificación
Comprobamos que train y test mantienen distribuciones similares de charges.

In [ ]:
# Comparar distribuciones
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(train_df['charges'], kde=True, ax=axes[0], bins=30, color='steelblue')
axes[0].set_title('Train - Distribución de Charges', fontweight='bold')
axes[0].set_xlabel('Charges ($)')

sns.histplot(test_df['charges'], kde=True, ax=axes[1], bins=30, color='coral')
axes[1].set_title('Test - Distribución de Charges', fontweight='bold')
axes[1].set_xlabel('Charges ($)')

plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, '07_train_test_distribution.png'), dpi=100)
plt.show()

print('=== ESTADÍSTICAS COMPARATIVAS ===')
print(f'Train - Media: ${train_df["charges"].mean():,.2f}, Desv Std: ${train_df["charges"].std():,.2f}')
print(f'Test  - Media: ${test_df["charges"].mean():,.2f}, Desv Std: ${test_df["charges"].std():,.2f}')
print(f'\nDiferencia de medias: ${abs(train_df["charges"].mean() - test_df["charges"].mean()):,.2f}')

> **Justificación**: Estratificar garantiza que las proporciones de charges en train y test sean similares, evitando sesgos en la evaluación del modelo. Las distribuciones deben verse prácticamente idénticas.

## 13. Validación de datos limpios
Verificamos que no hay duplicados ni valores faltantes en los splits finales.

In [ ]:
print('=== VALIDACIÓN TRAIN ===' )
print(f'Registros: {train_df.shape[0]}')
print(f'Valores faltantes: {train_df.isnull().sum().sum()}')
print(f'Duplicados: {train_df.duplicated().sum()}')

print('\n=== VALIDACIÓN TEST ===')
print(f'Registros: {test_df.shape[0]}')
print(f'Valores faltantes: {test_df.isnull().sum().sum()}')
print(f'Duplicados: {test_df.duplicated().sum()}')

print('\n✓ Datasets listos para modelado')

## 14. Guardado de datasets y reportes

In [ ]:
# Guardar datasets
train_df.to_csv(os.path.join(DATA_DIR, 'train.csv'), index=False)
test_df.to_csv(os.path.join(DATA_DIR, 'test.csv'), index=False)
print('✓ Archivos salvados: train.csv, test.csv')

# Guardar deducciones
with open(os.path.join(OUTPUT_DIR, 'deductions.txt'), 'w', encoding='utf-8') as f:
    f.write('=== DEDUCCIONES DEL EDA ===\n\n')
    f.write('UNIVARIANTE:\n')
    f.write('- charges es fuertemente sesgada a la derecha (outliers en costos altos)\n')
    f.write('- smoker tiene desequilibrio: 20% fuma, 80% no fuma\n')
    f.write('- edad distribuida uniformemente (18-64 años)\n')
    f.write('- region uniforme entre 4 categorías\n\n')
    f.write('BIVARIANTE:\n')
    f.write('- Fumadores pagan 3-4x más que no fumadores (VARIABLE MÁS IMPORTANTE)\n')
    f.write('- age: correlación fuerte (0.60) - relación cuadrática\n')
    f.write('- bmi: correlación moderada (0.41)\n')
    f.write('- sex y region: correlaciones muy débiles (<0.1)\n')
    f.write('- children: correlación casi nula (-0.08)\n\n')
    f.write('ACCIONES TOMADAS:\n')
    f.write(f'- Eliminados {df.shape[0] - df_filtered.shape[0]} outliers por IQR\n')
    f.write('- Aplicada transformación logarítmica a charges\n')
    f.write('- Split 80/20 estratificado por charges\n')

print('✓ Deducciones guardadas: deductions.txt')

    # Guardar correlaciones
with open(os.path.join(OUTPUT_DIR, 'correlations.txt'), 'w', encoding='utf-8') as f:
    f.write('=== CORRELACIONES CON CHARGES (VARIABLE OBJETIVO) ===\n\n')
    for var, corr_val in correlations.items():
        f.write(f'{var}: {corr_val:.4f}\n')

print('✓ Correlaciones guardadas: correlations.txt')

# Guardar resumen
with open(os.path.join(OUTPUT_DIR, 'summary.txt'), 'w', encoding='utf-8') as f:
    f.write('=== RESUMEN EDA SEGURO MÉDICO ===\n\n')
    f.write('DATASET ORIGINAL:\n')
    f.write(f'- Registros: {df.shape[0]}\n')
    f.write(f'- Características: {df.shape[1]}\n')
    f.write(f'- Valores faltantes: 0\n\n')
    f.write('DESPUÉS DE FILTRADO (IQR k=1.5):\n')
    f.write(f'- Registros: {df_filtered.shape[0]}\n')
    f.write(f'- Outliers removidos: {df.shape[0] - df_filtered.shape[0]}\n')
    f.write(f'- Porcentaje eliminado: {(df.shape[0] - df_filtered.shape[0])/df.shape[0]*100:.2f}%\n\n')
    f.write('SPLIT FINAL:\n')
    f.write(f'- Train: {train_df.shape[0]} registros (80%)\n')
    f.write(f'- Test: {test_df.shape[0]} registros (20%)\n\n')
    f.write('VARIABLES NUMÉRICAS:\n')
    f.write(f'- age: rango 18-64 años\n')
    f.write(f'- bmi: rango 15.96-53.13\n')
    f.write(f'- charges: rango ${train_df["charges"].min():,.2f} - ${train_df["charges"].max():,.2f}\n\n')
    f.write('VARIABLES CATEGÓRICAS:\n')
    f.write(f'- sex: male, female\n')
    f.write(f'- smoker: yes (fumadores), no\n')
    f.write(f'- region: southwest, southeast, northwest, northeast\n')
    f.write(f'- children: 0 a 5 hijos\n')

print('✓ Resumen guardado: summary.txt')
print('\n=== EDA COMPLETADO EXITOSAMENTE ===')

## 15. Siguientes pasos

Los datos están listos para modelado. Próximas etapas:

1. **Preprocesamiento**: Encoding de categorías (one-hot encoding para region, label encoding para smoker/sex)
2. **Feature Scaling**: Normalización de age, bmi para algoritmos sensibles a escala
3. **Modelos de Regresión**: Entrenar modelos para predecir charges
4. **Evaluación**: Métricas como MAE, RMSE, R² para validar rendimiento
5. **Interpretabilidad**: Analizar importancia de features en predicciones

## 📁 Archivos generados

**Datasets:**
- `data/train.csv` - 80% para entrenamiento
- `data/test.csv` - 20% para evaluación

**Reportes:**
- `output/deductions.txt` - Observaciones clave
- `output/correlations.txt` - Matriz de correlaciones
- `output/summary.txt` - Resumen estadístico

**Visualizaciones:**
- `output/figs/01_univariate_*.png` - Distribuciones individuales
- `output/figs/02_univariate_*.png` - Conteos categóricos
- `output/figs/03_charges_transformation.png` - Antes/después log
- `output/figs/04_bivariate_*.png` - Scatter plots
- `output/figs/05_bivariate_*.png` - Box plots
- `output/figs/06_correlation_matrix.png` - Heatmap
- `output/figs/07_train_test_distribution.png` - Comparación splits